<a href="https://colab.research.google.com/github/cxrlton/ML/blob/main/sentiment_analysis_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
!pip install beautifulsoup4

In [101]:
import torch
import requests
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bs4 import BeautifulSoup
import re #regex function

In [102]:
tokenizer  = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [103]:
tokens = tokenizer.encode('I am a bad person', return_tensors='pt')

In [104]:
tokens[0]

tensor([  101,   151, 10345,   143, 12428, 14150,   102])

In [105]:
tokenizer.decode(tokens[0])

'[CLS] i am a bad person [SEP]'

In [106]:
result = model(tokens)

In [107]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.0463,  1.6272,  0.1969, -1.9026, -2.2480]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [108]:
result.logits

tensor([[ 3.0463,  1.6272,  0.1969, -1.9026, -2.2480]],
       grad_fn=<AddmmBackward0>)

In [109]:
int(torch.argmax(result.logits)) + 1

1

## Collect Reviews

In [113]:
r = requests.get('https://www.yelp.com/biz/zaxbys-richardson-3')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [115]:
reviews[0]

"I've been to Zaxby's a few times, and I have to give this location credit for freshly made food, a clean restaurant, and great customer service!Guadalupe and Ronnie were exceptional, showing patience with our order and ensuring that everyone in our party of seven had everything we needed throughout our dining experience. The only thing I'd say is that the fry portion could've been bigger and had a little less salt.Thank you so much for a pleasant experience!"

#Load into DF and Score


In [116]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.array(reviews), columns=['review'])

In [117]:
df

,review
0,"I've been to Zaxby's a few times, and I have t..."
1,Fast food? Waited in line for ONE car...ONE......
2,The food here is so good! The restaurant is a...
3,"First time trying Zaxbys, honestly was pretty ..."
4,These are the most bland wings in Dallas no ex...
5,Decided to finally try zaxby today .kinda thou...
6,Yelp 100 Challenge 2023#45This location is loc...
7,"I haven't eaten here, I have checked out the m..."
8,I went there one time and was pleasantly surpr...
9,Amazing chicken and fries.I love the chicken c...


In [118]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1

In [119]:
sentiment_score(df['review'].iloc[1])

1

In [120]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [121]:
df

,review,sentiment
0,"I've been to Zaxby's a few times, and I have t...",5
1,Fast food? Waited in line for ONE car...ONE......,1
2,The food here is so good! The restaurant is a...,5
3,"First time trying Zaxbys, honestly was pretty ...",3
4,These are the most bland wings in Dallas no ex...,5
5,Decided to finally try zaxby today .kinda thou...,2
6,Yelp 100 Challenge 2023#45This location is loc...,3
7,"I haven't eaten here, I have checked out the m...",2
8,I went there one time and was pleasantly surpr...,5
9,Amazing chicken and fries.I love the chicken c...,5


In [123]:
df['review'].iloc[0]

"I've been to Zaxby's a few times, and I have to give this location credit for freshly made food, a clean restaurant, and great customer service!Guadalupe and Ronnie were exceptional, showing patience with our order and ensuring that everyone in our party of seven had everything we needed throughout our dining experience. The only thing I'd say is that the fry portion could've been bigger and had a little less salt.Thank you so much for a pleasant experience!"